In [1]:
class TimeoutError(Exception):
    pass

In [2]:
try:
    raise TimeoutError('timeout occurred')
except TimeoutError as ex:
    print(repr(ex))

TimeoutError('timeout occurred',)


In [5]:
import sys

try:
    raise TimeoutError('timeout occurred')
except Exception as ex:
    print(ex.args,ex.__traceback__)

('timeout occurred',) <traceback object at 0x110d0db08>


In [7]:
class ReadOnlyError(AttributeError):
    pass

try:
    raise ReadOnlyError('Account number is read-ony','BA10001')
except ReadOnlyError as ex:
    print(repr(ex))

ReadOnlyError('Account number is read-ony', 'BA10001')


In [8]:
class ReadOnlyError(AttributeError):
    pass

try:
    raise ReadOnlyError('Account number is read-ony','BA10001')
except AttributeError as ex:
    print(repr(ex))

ReadOnlyError('Account number is read-ony', 'BA10001')


In [9]:
class ReadOnlyError(AttributeError):
    pass

try:
    raise ReadOnlyError('Account number is read-ony','BA10001')
except BaseException as ex:
    print(repr(ex))

ReadOnlyError('Account number is read-ony', 'BA10001')


In [12]:
class WebScraperException(Exception):
    pass

class HTTPException(WebScraperException):
    pass

class InvalidUrlException(HTTPException):
    pass

class TimeoutException(HTTPException):
    pass

class PingTimeoutException(TimeoutException):
    pass

class LoadTimeoutException(TimeoutException):
    pass

class ParseException(WebScraperException):
    pass

In [15]:
try:
    raise PingTimeoutException('Ping to www... time out')
except WebScraperException as ex:
    print(repr(ex))

PingTimeoutException('Ping to www... time out',)


In [27]:
class APIException(Exception):
    pass

class ApplicationException(APIException):
    pass

class DBException(ApplicationException):
    pass

class DBConnectionError(DBException):
    pass

class ClientException(APIException):
    pass

class NotFoundError(ClientException):
    pass

class NotAuthorizedError(ClientException):
    pass

In [28]:
class Account:
    def __init__(self,account_id,account_type):
        self.account_id = account_id
        self.account_type = account_type

In [29]:
def lookup_account_by_id(account_id):
    if not isinstance(account_id, int) or account_id <=0:
        raise ClientException(f'Account number {account_id} is invalid')

    if account_id<100:
        raise DBConnectionError('Permanent failure connecting to database')
    elif account_id<200:
        raise NotAuthorizedError('User does not have permission to read the account')
    elif account_id<300:
        raise NotFoundError('Account not found')
    else:        return Account('account_id','Saving')

In [30]:
from http import HTTPStatus

In [35]:
def get_account(account_id):
    try:
        account = lookup_account_by_id(account_id)
    except ApplicationException as ex:
        return HTTPStatus.INTERNAL_SERVER_ERROR, str(ex)
    except NotFoundError as ex:
        return HTTPStatus.NOT.FOUND,'The account {} does not exist'.format(account_id)
    except NotAuthorizedError as ex:
        return HTTPStatus.UNAUTHORIZED,'You do not have the proper authorization'
    except ClientException as ex:
        return HTTPStatus.BAD_REQUEST,str(ex)
    else:
        return HTTPStatus.OK,{'id':account.account_id,'type':account.account_type}

In [36]:
get_account('abc')

(<HTTPStatus.BAD_REQUEST: 400>, 'Account number abc is invalid')

In [37]:
get_account(350)

(<HTTPStatus.OK: 200>, {'id': 'account_id', 'type': 'Saving'})

In [39]:
class APIException(Exception):
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'API exception occurred'
    user_err_msg = 'We are sorry.An unexpected error occurred on our end'
    
    def __init__(self,*args,user_err_msg = None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)
            
        if user_err_msg is not None:
            self.user_err_msg = user_err_msg

In [40]:
try:
    raise APIException('custom message...',10,20)
except APIException as ex:
    print(repr(ex))
    print(ex.user_err_msg)

APIException('custom message...', 10, 20)
We are sorry.An unexpected error occurred on our end


In [41]:
import json

class APIException(Exception):
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'API exception occurred'
    user_err_msg = 'We are sorry.An unexpected error occurred on our end'
    
    def __init__(self,*args,user_err_msg = None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)
            
        if user_err_msg is not None:
            self.user_err_msg = user_err_msg
            
    def to_json(self):
        err_object = {'status': self.http_status,'message':self.user_err_msg}
        return json.dumps(err_object)

In [42]:
try:
    raise APIException()
except APIException as ex:
    print(repr(ex))
    print(ex.to_json())

APIException('API exception occurred',)
{"status": 500, "message": "We are sorry.An unexpected error occurred on our end"}


In [50]:
from datetime import datetime

class APIException(Exception):
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'API exception occurred'
    user_err_msg = 'We are sorry.An unexpected error occurred on our end'
    
    def __init__(self,*args,user_err_msg = None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)
            
        if user_err_msg is not None:
            self.user_err_msg = user_err_msg
            
    def to_json(self):
        err_object = {'status': self.http_status,'message':self.user_err_msg}
        return json.dumps(err_object)
    
    def log_exception(self):
        exception = {
            "type":type(self).__name__,
            "http_status":self.http_status,
            "message":self.args[0] if self.args else self.internal_err_msg,
            "args":self.args[1:]
        }
        print(f'EXCEPTION: {datetime.utcnow().isoformat()}:{exception}')

In [51]:
try:
    raise APIException()
except APIException as ex:
    print(repr(ex))
    print(ex.to_json())

APIException('API exception occurred',)
{"status": 500, "message": "We are sorry.An unexpected error occurred on our end"}


In [52]:
class ApplicationException(APIException):
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "Generic server side exception."
    user_err_msg = "We are sorry. An unexpected error occurred on our end."
    
class DBException(ApplicationException):
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "Database exception."
    user_err_msg = "We are sorry. An unexpected error occurred on our end."
    
class DBConnectionError(DBException):
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "DB connection error."
    user_err_msg = "We are sorry. An unexpected error occurred on our end."
    
class ClientException(APIException):
    http_status = HTTPStatus.BAD_REQUEST
    internal_err_msg = "Client submitted bad request."
    user_err_msg = "A bad request was received."
    
class NotFoundError(ClientException):
    http_status = HTTPStatus.NOT_FOUND
    internal_err_msg = "Resource was not found."
    user_err_msg = "Requested resource was not found."

class NotAuthorizedError(ClientException):
    http_status = HTTPStatus.UNAUTHORIZED
    internal_err_msg = "Client not authorized to perform operation."
    user_err_msg = "You are not authorized to perform this request."

In [53]:
def lookup_account_by_id(account_id):
    if not isinstance(account_id, int) or account_id <=0:
        raise ClientException(f'Account number {account_id} is invalid',
                             f'account_id = {account_id}',
                             'type error - accout number not an integer')

    if account_id<100:
        raise DBConnectionError('Permanent failure connecting to database','db = db01')
    elif account_id<200:
        raise NotAuthorizedError('User does not have permission to read the account',f'account_id={account_id}')
    elif account_id<300:
        raise NotFoundError('Account not found',f'account_id={account_id}')
    else:
        return Account('account_id','Saving')

In [54]:
def get_account(account_id):
    try:
        account = lookup_account_by_id(account_id)
    except APIException as ex:
        ex.log_exception()
        return ex.to_json
    else:
        return HTTPStatus.OK,{'id':account.account_id,'type':account.account_type}

In [55]:
get_account('ABC')

EXCEPTION: 2020-03-28T08:50:34.539345:{'type': 'ClientException', 'http_status': <HTTPStatus.BAD_REQUEST: 400>, 'message': 'Account number ABC is invalid', 'args': ('account_id = ABC', 'type error - accout number not an integer')}


<bound method APIException.to_json of ClientException('Account number ABC is invalid', 'account_id = ABC', 'type error - accout number not an integer')>

In [56]:
get_account(50)

EXCEPTION: 2020-03-28T08:51:18.434996:{'type': 'DBConnectionError', 'http_status': <HTTPStatus.INTERNAL_SERVER_ERROR: 500>, 'message': 'Permanent failure connecting to database', 'args': ('db = db01',)}


<bound method APIException.to_json of DBConnectionError('Permanent failure connecting to database', 'db = db01')>

In [57]:
get_account(250)

EXCEPTION: 2020-03-28T08:51:25.827584:{'type': 'NotFoundError', 'http_status': <HTTPStatus.NOT_FOUND: 404>, 'message': 'Account not found', 'args': ('account_id=250',)}


<bound method APIException.to_json of NotFoundError('Account not found', 'account_id=250')>

In [58]:
class AppException(Exception):
    pass

class NegativeIntegerError(AppException):
    pass

In [59]:
def set_age(age):
    if age<0:
        raise NegativeIntegerError('age can not be negative')

In [61]:
try:
    set_age(-10)
except NegativeIntegerError as ex:
    print(repr(ex))

NegativeIntegerError('age can not be negtive',)


In [62]:
ex = NegativeIntegerError()

In [63]:
isinstance(ex,AppException)

True

In [64]:
isinstance(ex,ValueError)

False

In [65]:
def set_age(age):
    if age < 0:
        raise NegativeIntegerError('age can not be negative')


In [66]:
try:
    set_age(-10)
except NegativeIntegerError as ex:
    print(repr(ex))

NegativeIntegerError('age can not be negative',)
